In [1]:
import time
import random
import requests
import pandas as pd
import json
from pandas import json_normalize

In [2]:
class Job104Spider():
    def search(self, keyword, max_nun=10, filter_params=None, sort_type='符合度', is_sort_asc=False):
        # 產生搜尋結果
        jobs = []
        total_count = 0

        url = 'https://www.104.com.tw/jobs/search/list'
        # 這個url不是從網址來的，要到F12>XHR>Headers中查找，所以一定要最後面的list
        query = f'ro=0&kwop=7&keyword={keyword}&expansionType=area,spec,com,job,wf,wktm&mode=s&jobsource=2018indexpoc'
        if filter_params:
            # 加上篩選參數，要先轉換為 URL 參數字串格式
            query += ''.join([f'&{key}={value}' for key, value, in filter_params.items()])
            
        # 加上排序條件
        sort_dict = {
            '符合度': '1',
            '日期': '2',
            '經歷': '3',
            '學歷': '4',
            '應徵人數': '7',
            '待遇': '13',
        }
        
        # 結果排序
        sort_params = f"&order={sort_dict.get(sort_type, '1')}"
        sort_params += '&asc=1' if is_sort_asc else '&asc=0'
        query += sort_params
            
        # 設定headers，偽裝成真人搜尋，避免被網頁偵測到
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
            'Referer': 'https://www.104.com.tw/jobs/search/',
        }
        
        page = 1
        while len(jobs) < max_nun:
            params = f'{query}&page={page}'
            r = requests.get(url, params=params, headers=headers)
            if r.status_code != requests.codes.ok:
                print('請求失敗', r.status_code)
                data = r.json()
                print(data['status'], data['statusMsg'], data['errorMsg'])
                break

            data = r.json()
            total_count = data['data']['totalCount']
            jobs.extend(data['data']['list'])

            if (page == data['data']['totalPage']) or (data['data']['totalPage'] == 0):
                break
            page += 1
            time.sleep(random.uniform(3, 5))

        return total_count, jobs[:max_nun]

In [3]:
job104_spider = Job104Spider()

In [4]:
filter_params = {
    'area': '6001001000,6001016000',  # (地區) 台北市,高雄市
    # 's9': '1,2,4,8',  # (上班時段) 日班,夜班,大夜班,假日班
    # 's5': '0',  # 0:不需輪班 256:輪班
    # 'wktm': '1',  # (休假制度) 週休二日
    # 'isnew': '0',  # (更新日期) 0:本日最新 3:三日內 7:一週內 14:兩週內 30:一個月內
    # 'jobexp': '1,3,5,10,99',  # (經歷要求) 1年以下,1-3年,3-5年,5-10年,10年以上
    # 'newZone': '1,2,3,4,5',  # (科技園區) 竹科,中科,南科,內湖,南港
    # 'zone': '16',  # (公司類型) 16:上市上櫃 5:外商一般 4:外商資訊
    # 'wf': '1,2,3,4,5,6,7,8,9,10',  # (福利制度) 年終獎金,三節獎金,員工旅遊,分紅配股,設施福利,休假福利,津貼/補助,彈性上下班,健康檢查,團體保險
    # 'edu': '1,2,3,4,5,6',  # (學歷要求) 高中職以下,高中職,專科,大學,碩士,博士
    # 'remoteWork': '1',  # (上班型態) 1:完全遠端 2:部分遠端
    # 'excludeJobKeyword': '科技',  # 排除關鍵字
    # 'kwop': '1',  # 只搜尋職務名稱
    }

In [5]:
total_count, jobs= job104_spider.search('python', max_nun=10, filter_params=filter_params)

In [6]:
print('搜尋結果職缺總數：', total_count)

搜尋結果職缺總數： 3487


In [7]:
s1 = json.dumps(jobs)
d2 = json.loads(s1)
df = json_normalize(d2)

In [8]:
df

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,major,link.applyAnalyze,link.job,link.cust
0,1,10926100,Backend Engineer,Backend Engineer,1,1,6001001011,台北市南港區,市民大道 8 段 369 號 6 樓之 1,Glasnostic is developing a groundbreaking clou...,...,hotjob_chr,Backend Engineer,美商格拉思緹有限公司台灣分公司,121.6081769,25.0536377,0,"[資訊工程相關, 資訊管理相關]",//www.104.com.tw/jobs/apply/analysis/6i6ms?cha...,//www.104.com.tw/job/6i6ms?jobsource=hotjob_chr,//www.104.com.tw/company/1a2x6bkr44?jobsource=...
1,1,12359914,(院本部)資訊系統管理人員 (台北),(院本部)資訊系統管理人員 (台北),1,1,6001001005,台北市大安區,,1.維運文件編寫並配合資安法導入ISMS/PIMS服務。\n2.資訊安全及網路管理。\n3....,...,hotjob_chr,(院本部)資訊系統管理人員 (台北),財團法人國家實驗研究院,121.5433783,25.0249441,0,[],//www.104.com.tw/jobs/apply/analysis/7cwyy?cha...,//www.104.com.tw/job/7cwyy?jobsource=hotjob_chr,//www.104.com.tw/company/76sfnje?jobsource=hot...
2,1,12602486,【聯合招募】財務投資中後台幕僚人員,【聯合招募】財務投資中後台幕僚人員,1,1,6001001005,台北市大安區,,本職缺只接受國泰人壽徵才網站的履歷表，請立即至以下網址登錄您的資料：『https://bit...,...,hotjob_chr,【聯合招募】財務投資中後台幕僚人員,國泰人壽保險股份有限公司_總公司_國泰金控,121.5433783,25.0249441,0,"[商業及管理學科類, 財稅金融相關, 經濟學相關]",//www.104.com.tw/jobs/apply/analysis/7i452?cha...,//www.104.com.tw/job/7i452?jobsource=hotjob_chr,//www.104.com.tw/company/1jt7ol5?jobsource=hot...
3,1,11532062,【儲備幹部】財金儲備幹部(TA),【儲備幹部】財金儲備幹部(TA),1,1,6001001004,台北市松山區,,直接走進一流交易室，站上國際金融舞台\n1.接受輪調培育訓練，建立全面Treasury專業視...,...,hotjob_chr,【儲備幹部】財金儲備幹部(TA),玉山銀行_玉山商業銀行股份有限公司,121.54645992791139,25.058053471807607,0,"[數學及電算機科學學科類, 財稅金融相關, 一般商業學類]",//www.104.com.tw/jobs/apply/analysis/6v672?cha...,//www.104.com.tw/job/6v672?jobsource=hotjob_chr,//www.104.com.tw/company/13quahyo?jobsource=ho...
4,1,12052543,【PropTech數位人才招募】雲端數據工程師,【PropTech數位人才招募】雲端數據工程師,1,1,6001001007,台北市信義區,信義路五段100號2樓(110),【關於PropTech數位人才招募計劃】\n信義企業集團的各項產品與計畫正融入大數據、機器學...,...,hotjob_chr,【PropTech數位人才招募】雲端數據工程師,信義房屋股份有限公司,121.5655218,25.0326659,0,"[數學及電算機科學學科類, 資訊管理相關, 數理統計相關]",//www.104.com.tw/jobs/apply/analysis/76bsv?cha...,//www.104.com.tw/job/76bsv?jobsource=hotjob_chr,//www.104.com.tw/company/a9pjy3k?jobsource=hot...
5,1,12457745,Senior HQA Software Engineer,Senior HQA Software Engineer,1,1,6001001011,台北市南港區,三重路19之2號2樓,Why Verifone\n\nFor more than 30 years VeriFon...,...,hotjob_chr,Senior HQA Software Engineer,新加坡商惠爾訊科技股份有限公司台灣分公司(VeriFone Systems Pte Ltd ...,121.6130666,25.0565712,0,"[電機電子工程相關, 資訊工程相關]",//www.104.com.tw/jobs/apply/analysis/7f0gh?cha...,//www.104.com.tw/job/7f0gh?jobsource=hotjob_chr,//www.104.com.tw/company/wj1ynwo?jobsource=hot...
6,2,12607018,Python 助教~兼職,<em class='b-txt--highlight'>Python</em> 助教~兼職,2,2,6001001001,台北市中正區,忠孝西路一段102號10樓,徵求 [[[Python]]] 助教，需熟悉 [[[Python]]] 數據分析。\n\n【...,...,jolist_a_relevance,Python 助教~兼職,達內教育集團_上台科技股份有限公司,121.5125535,25.0471797,0,[],//www.104.com.tw/jobs/apply/analysis/7i7my?cha...,//www.104.com.tw/job/7i7my?jobsource=jolist_a_...,//www.104.com.tw/company/1a2x6bk220?jobsource=...
7,0,12471941,python工程師,<em class='b-txt--highlight'>python</em>工程師,1,1,6001001005,台北市大安區,潮州街108號5樓,工作內容： \r\n1. [[[Python]]]維運及系統開發\r\n2.熟悉[[[Pyt...,...,jolist_a_relevance,python工程師,英特拉金融科技股份有限公司,121.5259941,25.0288157,0,[資訊工程相關],//www.104.com.tw/jobs/apply/analysis/7fbet?cha...,//www.104.com.tw/job/7fbet?jobsource=jolist_a_...,//www.104.com.tw/company/1a2x6bkus5?jobsource=...
8,0,10039869,Python Engineer,<em class='b-txt--highlight'>Python</em> Engineer,1,1,6001001010,台北市內湖區,民權東路三段142號5樓506室,1.具備[[[Python]]]實務專案經驗\n2.有Vue開發框架佳\n3.太陽能系統開發...,...,jolist_a_relevance,Python Engineer,億力資訊股份有限公司,121.5470823,25.0617126,0,[],//www.104.com.tw/jobs/apply/analysis/5z6t9?cha...,//www.104.com.tw/job/5z6t9?jobsource=jolist_a_...,//www.104.com.tw/company/bjvssxk?jobsource=jol...
9,0,12077849,Python後端工程師,<em class='b-txt--highlight'>Python</em>後端工程師,1,1,6001001004,台北市松山區,,使用技術條件：\n熟悉 [[[Python]]] 程式語言\n熟悉 Django、Flask...,...,jolist_a_relevance,Python後端工程師,易勝資訊股份有限公司,121.5638621,25.0541591,0,"[資訊管理相關, 資訊工程相關, 其他數學及電算機科學相關]",//www.104.com.tw/jobs/apply/analysis/76vb